# Criando a máquina Preditiva

In [1]:
## Pacotes
import pandas as pd     #Biblioteca de Funções de Manipulação de Dados
import seaborn as sns   #Biblioteca de Funções de Visualização Gráfica
import warnings
warnings.filterwarnings("ignore")

#Importação dos Dados
clientes = pd.read_csv('/content/clientes.csv')

#Substituindo caracteres por números
clientes['sexo']= clientes['sexo'].map({'Male':0, 'Female':1})
clientes['estado_civil']= clientes['estado_civil'].map({'No':0, 'Yes':1})
clientes['aprovacao_emprestimo']= clientes['aprovacao_emprestimo'].map({'N':0, 'Y':1})

# Eliminando valores Nulos
clientes = clientes.dropna()

# Separando variáveis Explicativas e Variável TARGET
X = clientes[['sexo', 'estado_civil', 'renda', 'emprestimo', 'historico_credito']]
y = clientes.aprovacao_emprestimo
X.shape, y.shape

# importando Pacote para Amostragem dos Dados
from sklearn.model_selection import train_test_split

# Realizando Amostragem dosa Dados
x_train, x_teste, y_train, y_teste = train_test_split(X,y, test_size = 0.2, random_state = 7)

#Importando Pacotes para Construção das Máquinas Preditivas com Machine Learning

## Algoritmo Random Forest
from sklearn.ensemble import RandomForestClassifier 

#Importando a Métrica de Avaliação
from sklearn.metrics import accuracy_score

# Criando a Máquina Preditiva com o Random Forest 
maquina = RandomForestClassifier(max_depth=4, random_state = 7) 
maquina.fit(x_train, y_train)

#Avaliando a Acurácia da Máquina Preditiva Baseline com dados de Teste
pred_maquina_teste = maquina.predict(x_teste)
accuracy_score(y_teste,pred_maquina_teste)

#Comando de Salvamento da Máquina Preditiva
import pickle 
pickle_out = open("maquina_preditiva.pkl", mode = "wb") 
pickle.dump(maquina, pickle_out) 
pickle_out.close()

## 1° Instalar o Pacote para Conexão do nosso Sistema na WEB

In [2]:
#instalação do Pacote para Conexão do nosso Sistema na WEB 
!pip install -q pyngrok

## 2° Instalar o Pacote para Implementação do nosso Sistema na WEB

In [3]:
#instalação dos Pacotes para Implementação do nosso Sistema na WEB
!pip install -q streamlit
!pip install -q streamlit_ace

## 3° Criar o Sistema  

In [4]:
%%writefile app.py
#Comando para Criar o Sistema no arquivo app.py
# script python para ser executado de uma vez

#Importando Pacotes# 
import pickle
import streamlit as st
 
# Carregando a Máquina Preditiva
pickle_in = open('maquina_preditiva.pkl', 'rb') 
maquina_preditiva = pickle.load(pickle_in)

#Manter a sessão em cache 
@st.cache()
  
# Criando a função que irá fazer a predição usando os dados impostados pelo usuário do Sistema 
def prediction(sexo, estado_civil, renda, emprestimo, historico_credito):   
 
    # Pre-processando a entrada do Usuário    
    if sexo == "Masculino":
        sexo = 0
    else:
        sexo = 1
 
    if estado_civil == "Solteiro(a)":
        estado_civil = 0
    else:
        estado_civil = 1
 
    if historico_credito == "Débitos Pendentes":
        historico_credito = 0
    else:
        historico_credito = 1  
 
    emprestimo = emprestimo / 1000
 
    # Fazendo Predições
    prediction = maquina_preditiva.predict( 
        [[sexo, estado_civil, renda, emprestimo, historico_credito]])
     
    if prediction == 0:
        pred = 'Rejeitado'
    else:
        pred = 'Aprovado'
    return pred
      
  
# Essa função é para criação da webpage  
def main():  

    # Elementos da webpage
    # Nesse Ponto vc deve Personalizar o Sistema com sua Marca
    html_temp = """ 
    <div style ="background-color:blue;padding:13px"> 
    <h1 style ="color:white;text-align:center;">SAE</h1> 
    <h2 style ="color:white;text-align:center;">Sistema de Aprovação de Empréstimos - by Carlos Alberto Silva Júnior</h2> 
    </div> 
    """
      
    # Função do stramlit que faz o display da webpage
    st.markdown(html_temp, unsafe_allow_html = True) 
      
    # As linhas abaixo criam as caixas na qual o usuário vai entrar com dados da pessoa que quer o empréstimo para fazer a Predição
    sexo = st.selectbox('Sexo',("Masculino","Feminino"))
    estado_civil = st.selectbox('Estado Civil',("Solteiro(a)","Casado(a)")) 
    renda = st.number_input("Renda Mensal") 
    emprestimo = st.number_input("Valor Total do Empréstimo")
    historico_credito = st.selectbox('Histórico de Créditos',("Sem Débitos","Débitos Pendentes"))
    result =""
      
    #Quando o Usuário clicar no botão "Verificar" a Máquina Preditiva faz seu trabalho
    if st.button("Verificar"): 
        result = prediction(sexo, estado_civil, renda, emprestimo, historico_credito) 
        st.success('O empréstimo foi {}'.format(result))
        print(emprestimo)
     
if __name__=='__main__': 
    main()


Overwriting app.py


## 4° Executar o Sistema

In [5]:
#Executa o sistema
!streamlit run app.py &>/dev/null&

##  5° Criar a Conexão do nosso Sistema na WEB

In [6]:
# Importando a Biblioteca para Conexão do nosso Sistema na WEB 
from pyngrok import ngrok

## 6° Publicar nosso Sistema numa URL

In [7]:
#Publica o sistema numa URL
public_url = ngrok.connect('8501')

## 7° Abrir o Sistema via Link que contém a palavra ngrok.io

In [8]:
#URL do sistema
public_url

<NgrokTunnel: "http://e7e08e9c4881.ngrok.io" -> "http://localhost:8501">